In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict

## Data Preprocessing

### Read the preprocessed dataset:
 - "origin_dest_trips_census_tract_level.csv"
 - "tracts_4.csv"

In [2]:
# this dataset is for the nested hashmap
OD_p1_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p1.csv')[['RESIDENCE','WORKPLACE','JOBS']]
OD_p2_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p2.csv')[['RESIDENCE','WORKPLACE','JOBS']]
OD_p3_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p3.csv')[['RESIDENCE','WORKPLACE','JOBS']]
OD_p4_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p4.csv')[['RESIDENCE','WORKPLACE','JOBS']]

In [21]:
OD_p1_df

,RESIDENCE,WORKPLACE,JOBS
0,17001000100,17001000100,5005.177622
1,17001000100,17001000201,432.101665
2,17001000100,17001000202,324.076249
3,17001000100,17001000400,360.084721
4,17001000100,17001000500,4285.008180
...,...,...,...
80423,17203030700,17113005102,46.336570
80424,17203030700,17143001600,36.275552
80425,17203030700,17179021201,155.851852
80426,17203030700,17203030601,480.273810


In [3]:
ct_info_cases_data = pd.read_csv("Statistical_Analysis/data/ct_info_cases_data.csv")
ct_cases_df = ct_info_cases_data[['GEOID','countyFIPS','estimate_cases1','estimate_cases2','estimate_cases3','estimate_cases4']]
ct_cases_df

,GEOID,countyFIPS,estimate_cases1,estimate_cases2,estimate_cases3,estimate_cases4
0,17091011700,17091,0.030119,25.450775,36.353948,114.031521
1,17091011800,17091,0.023156,19.566633,27.949026,87.667780
2,17119400951,17119,0.018442,9.442116,15.933571,140.008883
3,17119400952,17119,0.012385,6.341011,10.700455,94.025297
4,17135957500,17135,0.000000,4.240201,4.783816,72.953196
...,...,...,...,...,...,...
3118,17037000100,17037,0.000000,25.211487,33.955725,174.629441
3119,17037001500,17037,0.000000,14.299781,19.259452,99.048612
3120,17037000400,17037,0.000000,32.385793,43.618334,224.322860
3121,17037000300,17037,0.000000,10.066565,13.558007,69.726892


### Create a dictionary: to record each origin->destination: number of trips

use the dataset: OD_census_tract_level 

1. Generate a unique set of origins in commute trips

2. Generate a doubly nested hashmap (python dictionary: key-value pair): \
 first mapping: **origin -> destination** \
 second mapping: **destination -> number of trips**

In [4]:
#function to fo defaultdict(defaultdict(int))
# when {} is empty, enable
# df['a']['b']+=1
def defaultdict_float():
    return defaultdict(float)

In [5]:
# anuallly aggregated data 
# origin -> destination (county level) frequency dictionary
# rename the origin-> destination df
OD = OD_p1_df

nu_OD_DblMap = defaultdict(defaultdict_float)
num_ct = OD_p1_df.shape[0]

for i in range(num_ct):
    fip_o = OD.iloc[i,0]
    fip_d = OD.iloc[i,1]
    num_trips = OD.iloc[i,2]   
    num_cases = float(ct_cases_df[ct_cases_df['GEOID'] == fip_o]['estimate_cases1'])
    if num_cases == 0:
        continue
    else:
        nu_OD_DblMap[fip_o][fip_d] = num_trips * num_cases

KeyboardInterrupt: 

In [7]:
de_OD_Map = defaultdict(float)
for fip_d in nu_OD_DblMap:
    sum = 0
    for fip_o in nu_OD_DblMap:
        sum += nu_OD_DblMap[fip_o][fip_d]
    de_OD_Map[fip_d] = sum  

In [8]:
Risk_DblMap = defaultdict(defaultdict_float)
for fip_o in nu_OD_DblMap:
    for fip_d in nu_OD_DblMap[fip_o]:
        nu = nu_OD_DblMap[fip_o][fip_d]
        de = de_OD_Map[fip_d]
        if de == 0 or nu == 0:
            continue
        else:               
            Risk_DblMap[fip_o][fip_d] = nu/de

In [6]:
# new
def getUnnormalizedRisk(OD, estimate_cases, apple_mobility):
    nu_OD_DblMap = defaultdict(defaultdict_float)
    num_ct = OD.shape[0]

    for i in range(num_ct):
        fip_o = OD.iloc[i,0]
        fip_d = OD.iloc[i,1]
        num_trips = OD.iloc[i,2]   
        num_cases = float(ct_cases_df[ct_cases_df['GEOID'] == fip_o][estimate_cases])
        if num_cases == 0:
            continue
        else:
            nu_OD_DblMap[fip_o][fip_d] = num_trips * num_cases * apple_mobility
    return nu_OD_DblMap

In [24]:
def getRisk(OD, estimate_cases, apple_mobility):      
    nu_OD_DblMap = defaultdict(defaultdict_float)
    num_ct = OD.shape[0]
    for i in range(num_ct):
        fip_o = OD.iloc[i,0]
        fip_d = OD.iloc[i,1]
        num_trips = OD.iloc[i,2]   
        num_cases = float(ct_cases_df[ct_cases_df['GEOID'] == fip_o][estimate_cases])
        if num_cases == 0:
            continue
        else:
            nu_OD_DblMap[fip_o][fip_d] = num_trips * num_cases * apple_mobility

    de_OD_Map = defaultdict(float)
    for fip_d in nu_OD_DblMap:
        sum = 0
        for fip_o in nu_OD_DblMap:
            sum += nu_OD_DblMap[fip_o][fip_d]
        de_OD_Map[fip_d] = sum  

    Risk_DblMap = defaultdict(defaultdict_float)
    for fip_o in nu_OD_DblMap:
        for fip_d in nu_OD_DblMap[fip_o]:
            nu = nu_OD_DblMap[fip_o][fip_d]
            de = de_OD_Map[fip_d]
            if de == 0 or nu == 0:
                continue
            else:               
                Risk_DblMap[fip_o][fip_d] = nu/de
    return Risk_DblMap

In [66]:
def getDPTotalRisk(OD, estimate_cases, apple_mobility):
    nu_OD_DblMap = defaultdict(defaultdict_float)
    num_ct = OD.shape[0]
    total_riskflow = 0

    for i in range(num_ct):
        fip_o = OD.iloc[i,0]
        fip_d = OD.iloc[i,1]
        num_trips = OD.iloc[i,2]   
        num_cases = float(ct_cases_df[ct_cases_df['GEOID'] == fip_o][estimate_cases])
        total_riskflow += num_trips * num_cases 
        if num_cases == 0:
            continue
        else:
            nu_OD_DblMap[fip_o][fip_d] = num_trips * num_cases
        
    for fip_o in nu_OD_DblMap:
        for fip_d in nu_OD_DblMap[fip_o]:
            if nu_OD_DblMap[fip_o][fip_d] == 0:
                continue
            else:
                nu_OD_DblMap[fip_o][fip_d] = nu_OD_DblMap[fip_o][fip_d]/total_riskflow * apple_mobility
    print(total_riskflow)       
    return nu_OD_DblMap

In [7]:
unor_Risk_DblMap1 =  getUnnormalizedRisk(OD = OD_p1_df, 
                       estimate_cases = 'estimate_cases1',
                       apple_mobility = 109.278621/100)
unor_Risk_DblMap2 =  getUnnormalizedRisk(OD = OD_p2_df, 
                       estimate_cases = 'estimate_cases2',
                       apple_mobility = 73.2133333/100)
unor_Risk_DblMap3 =  getUnnormalizedRisk(OD = OD_p3_df, 
                       estimate_cases = 'estimate_cases3',
                       apple_mobility = 130.970323/100)
unor_Risk_DblMap4 =  getUnnormalizedRisk(OD = OD_p4_df,
                       estimate_cases = 'estimate_cases4',
                       apple_mobility = 139.87/100)         

In [8]:
list_unor_Risk_DblMap = [unor_Risk_DblMap1, unor_Risk_DblMap2, unor_Risk_DblMap3, unor_Risk_DblMap4]

In [25]:
Risk_DblMap1 = getRisk(OD = OD_p1_df, 
                       estimate_cases = 'estimate_cases1',
                       apple_mobility = 109.278621/100)
Risk_DblMap2 = getRisk(OD = OD_p2_df, 
                       estimate_cases = 'estimate_cases2',
                       apple_mobility = 73.2133333/100)
Risk_DblMap3 = getRisk(OD = OD_p3_df, 
                       estimate_cases = 'estimate_cases3',
                       apple_mobility = 130.970323/100)
Risk_DblMap4 = getRisk(OD = OD_p4_df,
                       estimate_cases = 'estimate_cases4',
                       apple_mobility = 139.87/100)

In [26]:
list_Risk_DblMap = [Risk_DblMap1, Risk_DblMap2, Risk_DblMap3, Risk_DblMap4]

In [67]:
dptotal_Risk_DblMap1 =  getDPTotalRisk(OD = OD_p1_df, 
                                         estimate_cases = 'estimate_cases1',
                                         apple_mobility = 109.278621/100)
dptotal_Risk_DblMap2 =  getDPTotalRisk(OD = OD_p2_df,
                                         estimate_cases = 'estimate_cases2',
                                         apple_mobility = 73.2133333/100)
dptotal_Risk_DblMap3 =  getDPTotalRisk(OD = OD_p3_df,
                                         estimate_cases = 'estimate_cases3',
                                         apple_mobility = 130.970323/100)
dptotal_Risk_DblMap4 =  getDPTotalRisk(OD = OD_p4_df,
                                         estimate_cases = 'estimate_cases4',
                                         apple_mobility = 139.87/100)

8836264.208173046
1810276645.1046925
2281021844.448017
6247458949.558438


In [68]:
list_dptotal_Risk_DblMap = [dptotal_Risk_DblMap1, dptotal_Risk_DblMap2, 
                             dptotal_Risk_DblMap3, dptotal_Risk_DblMap4]

### Transfer double risk map to csv file and save to path

In [9]:
def saveRiskflowToCSV(Risk_DblMap, path):
    riskflow_df = pd.DataFrame(columns = ('START', 'END', 'RISK_FLOW'))
    for fip_o in Risk_DblMap:
        for fip_d in Risk_DblMap[fip_o]:
            riskflow = Risk_DblMap[fip_o][fip_d]
            new_row = pd.DataFrame({'START':[fip_o], 'END':[fip_d], 'RISK_FLOW': [riskflow] })
            riskflow_df = riskflow_df.append(new_row)
    riskflow_df.to_csv(path)

In [10]:
saveRiskflowToCSV(Risk_DblMap = unor_Risk_DblMap1, 
                  path = 'Streetlight_Data/clean_data/unnorm/unor_riskflow1_df.csv')
saveRiskflowToCSV(Risk_DblMap = unor_Risk_DblMap2, 
                  path = 'Streetlight_Data/clean_data/unnorm/unor_riskflow2_df.csv')
saveRiskflowToCSV(Risk_DblMap = unor_Risk_DblMap3, 
                  path = 'Streetlight_Data/clean_data/unnorm/unor_riskflow3_df.csv')
saveRiskflowToCSV(Risk_DblMap = unor_Risk_DblMap4, 
                  path = 'Streetlight_Data/clean_data/unnorm/unor_riskflow4_df.csv')

In [35]:
saveRiskflowToCSV(Risk_DblMap = Risk_DblMap1, 
                  path = 'Streetlight_Data/clean_data/riskflow1_df.csv')
saveRiskflowToCSV(Risk_DblMap = Risk_DblMap2, 
                  path = 'Streetlight_Data/clean_data/riskflow2_df.csv')
saveRiskflowToCSV(Risk_DblMap = Risk_DblMap3, 
                  path = 'Streetlight_Data/clean_data/riskflow3_df.csv')
saveRiskflowToCSV(Risk_DblMap = Risk_DblMap4, 
                  path = 'Streetlight_Data/clean_data/riskflow4_df.csv')

In [14]:
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap1, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow1_df.csv')
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap2, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow2_df.csv')
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap3, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow3_df.csv')
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap4, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow4_df.csv')

### Calculaye exposure entropy

In [ ]:
# Exposure_Map = defaultdict(defaultdict_float)
de_Risk_Map = defaultdict(float)
for fip_d in Risk_DblMap:
    sum = 0
    for fip_o in Risk_DblMap[fip_o]:
        sum += Risk_DblMap[fip_o][fip_d]
    de_Risk_Map[fip_d] = sum 

In [ ]:
reverse_Risk_DblMap = defaultdict(defaultdict_float)
for fip_o in Risk_DblMap:
    for fip_d in Risk_DblMap[fip_o]:
        risk = Risk_DblMap[fip_o][fip_d]   
        if risk == 0:
            continue
        else:
            reverse_Risk_DblMap[fip_d][fip_o] = risk

In [ ]:
np.sum(list(reverse_Risk_DblMap[17031811701].values()))

In [ ]:
Exposure_Map = {}
for fip_d in reverse_Risk_DblMap:
    v = np.array(list(reverse_Risk_DblMap[fip_d].values()))
    logv = np.log(v)
    entropy = - np.sum(v * logv)
    Exposure_Map[fip_d] = entropy

In [156]:
def getExposure(Risk_DblMap):
#     reverse_Risk_DblMap = defaultdict(defaultdict_float)
#     for fip_o in Risk_DblMap:
#         for fip_d in Risk_DblMap[fip_o]:
#             risk = Risk_DblMap[fip_o][fip_d]   
#             if risk == 0:
#                 continue
#             else:
#                 reverse_Risk_DblMap[fip_d][fip_o] = risk

    Exposure_Map = {}
    for fip_d in Risk_DblMap:
        unor_v = np.array(list(Risk_DblMap[fip_d].values()))
        Sigma = np.sum(unor_v)
        unor_v[unor_v == 0] = 10**-10
        v = unor_v/Sigma
        v = unor_v
        logv = np.log(v)
        entropy =  np.sum(v * logv)
        Exposure_Map[fip_d] = entropy
    return Exposure_Map

In [157]:
unor_Exposure_Map4 = getExposure(unor_Risk_DblMap4)
unor_Exposure_Map3 = getExposure(unor_Risk_DblMap3)
unor_Exposure_Map2 = getExposure(unor_Risk_DblMap2)
unor_Exposure_Map1 = getExposure(unor_Risk_DblMap1)

In [111]:
Risk_DblMap = unor_Risk_DblMap4
Exposure_Map = {}
for fip_d in Risk_DblMap:
    unor_v = np.array(list(Risk_DblMap[fip_d].values()))
    Sigma = np.sum(unor_v)
    unor_v[unor_v == 0] = 10**-10
    v = unor_v/Sigma
    logv = np.log(v)
    entropy = np.sum(v * logv)
    Exposure_Map[fip_d] = entropy

In [159]:
def saveExposureToCSV(list_Risk_DblMap, path):
    Exposure_Map1 = getExposure(list_Risk_DblMap[0])
    Exposure_Map2 = getExposure(list_Risk_DblMap[1])
    Exposure_Map3 = getExposure(list_Risk_DblMap[2])
    Exposure_Map4 = getExposure(list_Risk_DblMap[3])
    
    exposure_df = pd.DataFrame(columns = ('FIPS', 'exposure1', 'exposure2', 'exposure3', 'exposure4'))
    
    for i, fip_d in enumerate(Exposure_Map1): 
        FIPS = str(fip_d)
        exposure1 = Exposure_Map1[fip_d]
        exposure2 = Exposure_Map2[fip_d]
        exposure3 = Exposure_Map3[fip_d]
        exposure4 = Exposure_Map4.get(fip_d, 0)
        
        new_row = pd.DataFrame({'FIPS':[FIPS], 'exposure1':[exposure1],
                            'exposure2':[exposure2], 'exposure3':[exposure3],
                            'exposure4':[exposure4]})
        exposure_df = exposure_df.append(new_row)
    
    exposure_df.to_csv(path)
    

In [160]:
saveExposureToCSV(list_unor_Risk_DblMap, 'Streetlight_Data/clean_data/unnorm/unor_exposure_df.csv')

In [147]:
saveExposureToCSV(list_Risk_DblMap, 'Streetlight_Data/clean_data/risk/exposure_df.csv')

NameError: name 'list_Risk_DblMap' is not defined

In [72]:
saveExposureToCSV(list_dptotal_Risk_DblMap, 'Streetlight_Data/clean_data/dptotal/dptotal_exposure_df.csv')

---

In [162]:
def getInlfowExposure(Risk_DblMap):
    reverse_Risk_DblMap = defaultdict(defaultdict_float)
    for fip_o in Risk_DblMap:
        for fip_d in Risk_DblMap[fip_o]:
            risk = Risk_DblMap[fip_o][fip_d]   
            if risk == 0:
                continue
            else:
                reverse_Risk_DblMap[fip_d][fip_o] = risk
    Exposure_Map = {}
    for fip_d in Risk_DblMap:
        unor_v = np.array(list(Risk_DblMap[fip_d].values()))
        Sigma = np.sum(unor_v)
        unor_v[unor_v == 0] = 10**-10
        v = unor_v/Sigma
        v = unor_v
        logv = np.log(v)
        entropy =  np.sum(v * logv)
        Exposure_Map[fip_d] = entropy
    return Exposure_Map

In [166]:
def saveInlfowExposureToCSV(list_Risk_DblMap, path):
    Exposure_Map1 = getInlfowExposure(list_Risk_DblMap[0])
    Exposure_Map2 = getInlfowExposure(list_Risk_DblMap[1])
    Exposure_Map3 = getInlfowExposure(list_Risk_DblMap[2])
    Exposure_Map4 = getInlfowExposure(list_Risk_DblMap[3])
    
    exposure_df = pd.DataFrame(columns = ('FIPS', 'inflow_exposure1', 'inflow_exposure2', 'inflow_exposure3', 'inflow_exposure4'))
    
    for i, fip_d in enumerate(Exposure_Map1): 
        FIPS = str(fip_d)
        exposure1 = Exposure_Map1[fip_d]
        exposure2 = Exposure_Map2[fip_d]
        exposure3 = Exposure_Map3[fip_d]
        exposure4 = Exposure_Map4.get(fip_d, 0)
        
        new_row = pd.DataFrame({'FIPS':[FIPS], 
                                'inflow_exposure1':[exposure1], 'inflow_exposure2':[exposure2],
                                'inflow_exposure3':[exposure3], 'inflow_exposure4':[exposure4]})
        exposure_df = exposure_df.append(new_row)
    
    exposure_df.to_csv(path)
    

In [167]:
saveInlfowExposureToCSV(list_unor_Risk_DblMap, 'Streetlight_Data/clean_data/unnorm/unor_inflow_exposure_df.csv')